In [55]:
import sys

sys.path.append('/Users/leportella/src/serenata/medidor-de-poder/api/candidates')

from decouple import config
from datetime import datetime
import pandas as pd
import mongoengine as me

from models.sessions import Sessions, SessionsByYear
from models import Candidates

me.connect('serenata')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [36]:
presences = pd.read_csv('../data/2017-05-29-presences.xz')

In [37]:
presences['date'] = pd.to_datetime(presences.date)
presences['year'] = presences.date.apply(lambda x: x.year)

In [38]:
presences['congressperson_name'] = presences.congressperson_name.apply(lambda x: x.split('-')[0].upper())

In [39]:
presences.head()

,term,congressperson_document,congressperson_name,party,state,date,present_on_day,justification,session,presence,year
0,55,361,GOULART,PSD,SP,2015-02-01 10:33:09,Present,NaN,SESSÃO PREPARATÓRIA Nº 001 - 01/02/2015,Present,2015
1,55,361,GOULART,PSD,SP,2015-02-01 10:33:09,Present,NaN,SESSÃO PREPARATÓRIA Nº 002 - 01/02/2015,Present,2015
2,55,361,GOULART,PSD,SP,2015-02-03 08:00:00,Present,NaN,ORDINÁRIA Nº 001 - 03/02/2015,Present,2015
3,55,361,GOULART,PSD,SP,2015-02-03 08:00:00,Present,NaN,EXTRAORDINÁRIA Nº 002 - 03/02/2015,Present,2015
4,55,361,GOULART,PSD,SP,2015-02-04 08:00:01,Present,NaN,ORDINÁRIA Nº 003 - 04/02/2015,Present,2015


In [40]:
presences.present_on_day.value_counts()

Present              205252
Justified absence     17017
Absent                 5418
Present (~)             265
Name: present_on_day, dtype: int64

In [41]:
presences['was_present'] = presences.present_on_day.apply(lambda x: 1 if x in ['Present', 'Present (~)'] else 0)
presences['was_absent'] = presences.present_on_day.apply(lambda x: 1 if x in ['Absent', 'Justified absence'] else 0)
presences['was_absent_with_justification']= presences.present_on_day.apply(lambda x: 1 if x=='Justified absence' else 0)

In [45]:
dd = presences[presences.was_absent_with_justification>0].groupby(
    ['congressperson_document', 'year']
 )['was_absent_with_justification'].count().unstack().fillna(0)

dd.head()

year,2015,2016,2017
congressperson_document,,,
1,21.0,18.0,6.0
2,0.0,22.0,0.0
3,8.0,0.0,0.0
4,13.0,22.0,7.0
5,45.0,34.0,9.0


In [53]:
def get_presence_count_by_year(df, congressperson_document, year):
    return df[(df.congressperson_document==congressperson_document) & (df.year==year) & (df.was_present>0)].shape[0]

def get_abstences_count_by_year(df, congressperson_document, year):
    return df[(df.congressperson_document==congressperson_document) & (df.year==year) & (df.was_absent>0)].shape[0]

def get_justification_count_by_year(df, congressperson_document, year):
    return df[(df.congressperson_document==congressperson_document) & (df.year==year) & (df.was_absent_with_justification>0)].shape[0]

In [62]:
for congressperson_document in presences.congressperson_document.unique():
    
    try:
        candidate = Candidates.objects.get(
            deputy_info__terms__match={'congressperson_document': str(congressperson_document),
                                       'begin': 2015}
        )
        print('Found: {}'.format(congressperson_document))
    except:
        print('Not Found: {}'.format(congressperson_document))
        continue
        
    data = {}
    for year in presences['year'].unique():
        sessions_present = get_presence_count_by_year(presences, congressperson_document, year)
        sessions_absent = get_abstences_count_by_year(presences, congressperson_document, year)
        sessions_absent_with_justification = get_justification_count_by_year(presences, congressperson_document, year)      
        total_sessions = sessions_present + sessions_absent
        
        sessions_present_percent = 0
        sessions_abscent_percent = 0
        sessions_abscent_justified_percent = 0
        
        if total_sessions:
            sessions_present_percent = sessions_present / total_sessions
            sessions_abscent_percent = sessions_absent / total_sessions
        if sessions_absent:
            sessions_abscent_justified_percent = sessions_absent_with_justification / sessions_absent
            
        sessions_by_year = SessionsByYear(                
            total_sessions=total_sessions,
            present=sessions_present,
            abstent=sessions_absent,
            abstent_with_justification=sessions_absent_with_justification,
            present_percent=sessions_present_percent,
            abstent_percent=sessions_abscent_percent,
            abstent_with_justification_percent=sessions_abscent_justified_percent
        )
        
        sessions = Sessions()
        byyear = getattr(sessions, 'year_{}'.format(year))
        byyear = sessions_by_year
        
        candidate.sessions_presence = sessions

Found: 361
Found: 19
Found: 268
Found: 412
Found: 368
Found: 251
Found: 441
Found: 359
Found: 428
Found: 111
Found: 141
Found: 60
Found: 318
Found: 62
Found: 451
Not Found: 384
Found: 120
Found: 456
Found: 152
Found: 381
Found: 233
Found: 258
Found: 403
Found: 66
Not Found: 72
Found: 57
Found: 424
Found: 129
Not Found: 452
Found: 475
Found: 182
Found: 325
Found: 472
Found: 281
Found: 175
Not Found: 397
Found: 429
Found: 307
Found: 211
Found: 320
Found: 125
Not Found: 286
Not Found: 179
Not Found: 181
Found: 507
Found: 444
Found: 402
Found: 469
Found: 333
Found: 396
Found: 411
Found: 432
Found: 16
Found: 193
Found: 482
Not Found: 63
Found: 192
Found: 283
Found: 59
Found: 230
Found: 349
Found: 414
Found: 95
Found: 490
Found: 219
Not Found: 38
Found: 7
Found: 335
Not Found: 415
Found: 3
Not Found: 379
Found: 292
Found: 249
Not Found: 114
Found: 383
Found: 161
Found: 33
Found: 253
Found: 404
Found: 64
Not Found: 68
Not Found: 427
Found: 446
Found: 6
Found: 128
Not Found: 227
Found: 338
Fou

In [63]:
presences.sample(10)

,term,congressperson_document,congressperson_name,party,state,date,present_on_day,justification,session,presence,year,was_present,was_absent,was_absent_with_justification
184496,55,438,VANDER LOUBET,PT,MS,2017-04-05 07:48:58,Present,NaN,EXTRAORDINÁRIA Nº 063 - 05/04/2017,Present,2017,1,0,0
53085,55,496,MARCO MAIA,PT,RS,2016-08-02 07:56:56,Present,NaN,EXTRAORDINÁRIA Nº 185 - 02/08/2016,Present,2016,1,0,0
102161,55,296,CRISTIANE BRASIL,PTB,RJ,2015-10-28 08:00:00,Justified absence,Health Care Leave,ORDINÁRIA Nº 329 - 28/10/2015,Absent,2015,0,1,1
27561,55,59,WHERLES ROCHA,PSDB,AC,2016-04-04 07:48:04,Present,NaN,EXTRAORDINÁRIA Nº 066 - 04/04/2016,Present,2016,1,0,0
189834,55,56,JÉSSICA SALES,PMDB,AC,2015-02-09 08:00:02,Present,NaN,EXTRAORDINÁRIA Nº 007 - 09/02/2015,Present,2015,1,0,0
129394,55,345,ELI CORREA FILHO,DEM,SP,2016-11-30 07:53:45,Present,NaN,ORDINÁRIA Nº 315 - 30/11/2016,Present,2016,1,0,0
191609,55,223,BRUNNY,PTC,MG,2016-08-22 07:55:53,Present,NaN,EXTRAORDINÁRIA Nº 203 - 22/08/2016,Present,2016,1,0,0
83918,55,153,LUCIANA SANTOS,PCdoB,PE,2017-05-24 07:43:22,Present,NaN,EXTRAORDINÁRIA Nº 131 - 24/05/2017,Present,2017,1,0,0
31600,55,7,REMÍDIO MONAI,PR,RR,2017-04-25 07:42:39,Present,NaN,EXTRAORDINÁRIA Nº 089 - 25/04/2017,Present,2017,1,0,0
200105,55,499,BOHN GASS,PT,RS,2016-07-06 07:57:49,Present,NaN,ORDINÁRIA Nº 170 - 06/07/2016,Present,2016,1,0,0
